# Trigger Detection Model

## Importing Model

Huge credits to Chris10M on github!!

In [6]:
import sys
import os
print(f"Kernel executable: {sys.executable}")
print(os.getcwd())

Kernel executable: /Users/sarath/Downloads/Projects/camhack/Vision/armVenv/bin/python
/Users/sarath/Downloads/Projects/camhack/Vision/Model Deployment


In [8]:
from triggerModel import EyeTrigger

torch_weights_path = './Weights/torch_weights.pth'
torch_blueprint = "./Weights/my_model.onnx"
landmark_detection_path = "./Weights/68_face_landmarks_predictor.dat"
triggerModel = EyeTrigger(torch_blueprint, torch_weights_path, landmark_detection_path)

## Side Quest: Port model from `keras` to `torch`

In [5]:
from keras.models import load_model
import torch
from onnx2pytorch import ConvertModel
import tf2onnx 
import onnx
from tensorflow import TensorSpec, float32

In [ ]:
weights_path = "./Vision/open-eye-detector/models/weights.149-0.01.hdf5"
model = load_model(weights_path, compile = False)
input_shape = (1,10,20,1)
spec = (TensorSpec(input_shape, float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model,input_signature= spec, opset=13)

with open("my_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [8]:
onnx_model = onnx.load("my_model.onnx")
pytorch_model = ConvertModel(onnx_model)
pytorch_model.eval()

output_pytorch_path = '/Users/sarath/Downloads/Projects/camhack/Vision/Model Deployment/torch_weights.pth'
torch.save(pytorch_model.state_dict(), output_pytorch_path)


# Inference Testing

In [4]:
import cv2 as cv
from cameraController import CameraController

In [1]:
cam = cv.VideoCapture(1)

if not cam.isOpened():
    raise EnvironmentError("Camera was not able to be opened.")

controller = CameraController(cam)
try:
    while True:
        frameData = controller.getFrame()
        print(triggerModel.eyesOpen(*frameData))

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cam.release()
    cv.destroyAllWindows()

NameError: name 'cv' is not defined